# 02-01 SQL-Demo

## Hinweise zur Übung

Ziel dieser Demo ist eine Übersicht der wesentlichen SQL-Statements & -Vorgehensweisen, um Abfragen auf den Daten unseres Beispiel-DWH erstellen zu können.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql

In [ ]:
import os
import sys
import urllib.request
import gzip
import shutil
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
# Konfiguration
base_url_quellen   = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/08-Datenanalyse+Visualisierung/data"
base_url_reporting = "./"

In [ ]:
# SQlite-Datenbanken aus Github auf den Jupyter-Server herunterladen
urllib.request.urlretrieve(base_url_quellen + "/dwh/reporting.sqlite.gz", base_url_reporting + "reporting.sqlite.gz")

# Die Sqlite-Datenbank ist aufgrund ihrer Größe gezipped und muss vor der Nutzung noch entpackt werden
with gzip.open(base_url_reporting + "reporting.sqlite.gz", "rb") as f_in:
    with open(base_url_reporting + "reporting.sqlite", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_reporting      = base_url_reporting + "reporting.sqlite"

db_url_reporting      = "sqlite:///" + db_path_reporting

%sql $db_url_reporting

## Orientierung in der Datenbank

Im Kurs nutzen wir die Datenbank *SQLite*, die dateibasiert arbeitet. Sie erfordert keinen dedizierten Server, wie andere Datenbanken (z.B. MySQL, Oracle) und ist daher "klein" genug, um auf embedded devices (z.B. Handies, Wearables) oder innerhalb eines Jupyter Notebook zu laufen. Im Gegenzug ist der Funktionsumfang teilweise gegenüber dem SQL-Standard bzw. Features größerer Datenbanken reduziert.

Mit den folgenden Kommandos können Sie sich einen Überblick über die Struktur der Datenbank und ihrer Tabellen machen. In SQLite wird hierzu das `PRAGMA`-Statement verwendet:

Mit `PRAGMA table_list` können Sie eine Liste der in der Datenbank enthaltenen Tabellen und der Anzahl ihrer Spalten bekommen:

In [ ]:
%%sql $db_url_reporting
PRAGMA table_list

Mit `PRAGMA TABLE_INFO(<Tabellenname>)` können Sie die Definition einer Tabelle mit ihren Spaltennamen & Datentypen sehen.

In [ ]:
%%sql $db_url_reporting
PRAGMA TABLE_INFO(f_faelle)

## Einfache SQL-Abfragen auf einer Tabelle

In den folgenden Codeblöcken werden einfache SQL-Abfragen auf einer einzelnen Tabelle (ohne Joins) gezeigt

### Abfrage aller Spalten einer Tabelle

Mit `SELECT * FROM <Tabtelle>` werden alle Spalten und Zeilen einer Tabelle ausgelesen. Um zu vermeiden, dass bei großen Tabellen der gesamte Inhalt ausgelesen und ggf. die Kapazität des Notebooks gesprengt wird, kann man mit der Klausel `LIMIT <Anzahl Datensätze>` die Zahl der zurückgelieferten Tabellenzeilen angeben.

*Das folgende Statement liest 10 Datensätze (alle Spalten) aus der Tabelle `D_PATIENT` aus:*

In [ ]:
%%sql $db_url_reporting
SELECT *
  FROM d_patient
 LIMIT 10

### Nur ausgewählte Spalten auslesen

Statt dem `*` können auch ausgewählte Spalten angegeben werden, die ausgelesen werden sollen (um nur tatsächlich benötigte Spalten zu bekommen oder z.B. die Reihenfolge der auszugebenden Spalten explizit vorzugeben).

*Das folgende Statement liest nur Nachname, Geburtsdatum und Geschlecht aus (auf 10 Datensätze begrenzt):*

In [ ]:
%%sql $db_url_reporting
SELECT patient_nachname, patient_gebdat, patient_geschlecht
  FROM d_patient
 LIMIT 10

### Nur ausgewählte Zeilen einer Tabelle auslesen

Mit einer `WHERE`-Klausel kann die Ausgabe auf Datensätze beschränkt werden, die eine oder mehrere Bedingungen erfüllen.

*Das folgende Statement liest nur Patienten mit weiblichem Geschlecht aus (begrenzt auf 10 Zeilen):*

In [ ]:
%%sql $db_url_reporting
SELECT patient_nachname, patient_gebdat, patient_geschlecht
  FROM d_patient
 WHERE patient_geschlecht = 'F'
 LIMIT 10

### Bedingungen mit Wildcards und Teilstrings formulieren

In der `WHERE`-Klausel kann statt mit dem Gleichheitsoperator `=` auch der `LIKE`-Operator verwendet werden, der "unscharfe" Suchen nach Teilstrings ermöglicht. Die zu verwendenden Wildcards können sich je nach Datenbank unterscheiden, sind aber häufig `_` für ein einzelnes Zeichen und `%` für eine beliebige Anzahl von Zeichen, die außerhalb des gesuchten Textes stehen können. Strings werden in SQL-Abfragen immer in einfache Anführungsstriche `'`gesetzt.

* `... WHERE patient_ort LIKE 'Bielefeld%'`: findet alle Texte, die mit "Bielefeld" beginnen, also z.B. "Bielefeld-Sennestadt", nicht jedoch "West-Bielefeld"
* `... WHERE patient_ort LIKE '%Bielefeld'`: findet alle Texte, die mit Bielefeld enden, also z.B. "West-Bielefeld", aber nicht "Bielefeld-Sennestadt"
* `... WHERE patient_ort LIKE '%Bielefeld%'`: findet alle Texte, die das Wort "Bielefeld" enthalten, also beide der in den vorherigen Zeilen beschriebenen Fälle
* `... WHERE patient_ort LIKE 'Bielefel_'`: findet alle Texte, die mit "Bielefel" beginnen und danach ein einzelnes beliebiges Zeichen haben (also z.B. "Bielefel**d**" und "Bielefel**t**", nicht jedoch Bielefel"**dt**")

Datenbanken können je nach Konfiguration case-sensitive oder case-insensitive sein, also Groß-Kleinschreibung bei der Suche anwenden, oder nicht. Um Groß-Kleinschreibung als Störfaktor auszuschließen, können Texte mit den `LOWER()`- bzw. `UPPER()`-Funktionen in Klein- bzw. Großbuchstaben konvertiert werden.

* `... WHERE LOWER(patient_ort) = 'bielefeld'`: findet sowohl "Bielefeld" als auch "bielefeld" oder alle anderen möglichen Groß-Kleinschreibungen dieses Worts

Mit der `SUBSTR(<Spalte>, <erstes Zeichen>, <Länge>)`-Funktion können Teile eines Strings herausgeschnitten werden. Das erste Zeichen steht auf Position 1 (nicht 0).

* `SUBSTR(patient_gebdat, 1, 4)`: schneidet die ersten vier Zeichen des Geburtsdatums (hier: das Jahr) heraus



*Nur Patienten auslesen, deren Vorname mit 'Jon' beginnt:*

In [ ]:
%%sql $db_url_reporting
SELECT patient_nachname, patient_vorname, patient_gebdat, patient_ort
  FROM d_patient
 WHERE patient_vorname LIKE 'Jon%'
 LIMIT 10

*Nur Patienten auslesen, die im Jahr 1984 geboren sind:*

In [ ]:
%%sql $db_url_reporting
SELECT patient_nachname, patient_vorname, patient_gebdat, patient_ort
  FROM d_patient
 WHERE SUBSTR(patient_gebdat, 1, 4) = '1984'
 LIMIT 10

### Bedingen miteinander kombinieren

Kriterien in der `WHERE`-Klausel können mit Bool'scher Logik (`AND, OR, NOT`) verbunden und bei Bedarf geklammert werden. Besonders bei Kombination von `AND` und `OR` muss in der Regel geklammert werden, da `AND` "enger bindet" und in der Regel unerwünschte Kombinationen von Kriterien angewendet werden

*Nur Patienten auslesen, die im Jahr 1984 geboren sind und aus Bielefeld kommen:*

In [ ]:
%%sql $db_url_reporting
SELECT patient_nachname, patient_vorname, patient_gebdat, patient_ort
  FROM d_patient
 WHERE SUBSTR(patient_gebdat, 1, 4) = '1984'
   AND patient_ort = 'Bielefeld'
 LIMIT 10

*Nur Patienten auslesen, aus Bielefeld kommen und 1984 geboren sind oder aus Mannheim kommen und 1990 geboren sind (=> da AND enger bindet als OR, wird hier richtig ausgewertet):*

In [ ]:
%%sql $db_url_reporting
SELECT patient_nachname, patient_vorname, patient_gebdat, patient_ort, patient_geschlecht
  FROM d_patient
 WHERE patient_ort = 'Bielefeld'
   AND SUBSTR(patient_gebdat, 1, 4) = '1984'
    OR patient_ort = 'Mannheim'
   AND SUBSTR(patient_gebdat, 1, 4) = '1990'
 LIMIT 10

*Nur Patienten auslesen, aus Bielefeld kommen und 1984 oder 1990 geboren sind
(=> ohne Klammerung ist das Ergebnis falsch, da das Kriterium "Geburtsjahr=1990" unabhängig
vom Kriterium "Ort=Bielefeld" angewendet wird (unerwartet werden auch Patienten aus Mannheim ausgegeben))*

In [ ]:
%%sql $db_url_reporting
SELECT patient_nachname, patient_vorname, patient_gebdat, patient_ort, patient_geschlecht
  FROM d_patient
 WHERE patient_ort = 'Bielefeld'
   AND SUBSTR(patient_gebdat, 1, 4) = '1984'
    OR SUBSTR(patient_gebdat, 1, 4) = '1990'

*Nur Patienten auslesen, aus Bielefeld kommen und 1984 oder 1990 geboren sind
(=> Mit Klammerung werden beide Varianten für das Geburstjahr gemeinsam abgefragt und dann mit dem Ort verknüpft
(Ausgabe nur von Bielefelder Patienten aus beiden Geburtsjahren))*

In [ ]:
%%sql $db_url_reporting
SELECT patient_nachname, patient_vorname, patient_gebdat, patient_ort, patient_geschlecht
  FROM d_patient
 WHERE patient_ort = 'Bielefeld'
   AND (   SUBSTR(patient_gebdat, 1, 4) = '1984'
        OR SUBSTR(patient_gebdat, 1, 4) = '1990')

### Eindeutige Datensätze auslesen

Mit dem `DISTINCT`-Keyword werden identische Datensätze zusammenführt, so dass nur noch eindeutige Datensätze ausgegeben werden. Das ist z.B. dann hilfreich, wenn für eine Spalte eine Liste der darin genutzten Ausprägungen benötigt wird.

### Datensätze sortieren

Mit der `ORDER BY`-Klausel am Ende eines SQL-Statements (aber vor einer möglichen `LIMIT`-Klausel) können die abgefragten Datensätze sortiert werden.

Es kann dabei nach einer oder mehreren Spalten sowie auch nach abgeleiteten (z.B. berechneten oder aggregierten) Werten sortiert werden. Die Sortierung kann mit dem `DESC`-Keyword auch absteigend erfolgen. Je nach Datenbank ist es möglich, Werte alternativ zum Spaltennamen auch über vergebene Aliase oder die Reihenfolge der Spalten im `SELECT`-Abschnitt anzusprechen. Bei manchen Datenbanken müssen berechnete oder aggregierte Spalten in der `ORDER BY`-Klausel explizit wiederholt werden, wenn sie nicht über die vergebenen Aliase ansprechbar sind.

*Fallarten absteigend alphabetisch sortiert ausgeben:*

In [ ]:
%%sql $db_url_reporting
SELECT *
  FROM d_fallart
 ORDER BY fallart_name DESC

*Eindeutige Ausprägungen der Spalte patient_geschlecht in der Tabelle D_PATIENT auslesen:*

In [ ]:
%%sql $db_url_reporting
SELECT DISTINCT
       patient_geschlecht
  FROM d_patient

## Gruppieren und aggregieren von Tabelleninhalten

Mit der `GROUP BY`-Klausel können Spalten angegeben werden, die als Aggregationsebene genutzt werden sollen. Es muss dann immer auch eine Aggregatfunktion in der `SELECT`-Klausel der Abfrage angegeben werden. Wenn keine Aggregationsebene angegeben wird, wendet die Datenbank die Aggregatfunktionen auf die ganze Tabelle an.

Häufig verwendete Aggregatfunktionen sind:
* `COUNT(*)`: zählt die Anzahl der Datensätze innerhalb der Aggregationgsebene
* `COUNT(DISTINCT <Spalte>)`: Anzahl eindeutiger Ausprägungen in der angegebenen Spalte ermitteln
* `SUM(<Spalte>)`: summiert eine numerische Spalte auf
* `AVG(<Spalte>)`: bildet den Mittelwert einer numerischen Spalte innerhalb der Aggregationsebene
* `MIN(<Spalte)`: gibt den kleinsten Wert einer Spalte innerhalb der Aggregationsebene aus (bei Texten der an den Anfang sortierte)
* `MAX(<Spalte>)`: gibt den größten Wert einer Spalte innerhalb der Aggregationsebene aus (bei Texten der ans Ende sortierte)

Mit der `AS`-Klausel können Spalten zur Laufzeit der Abfrage umbenannt werden. Das macht besonders bei den Aggregatfunktionen Sinn, da sie berechnete Werte sind und so keinen eigenen Namen haben (die Datenbank setzt dann z.B. die Aggregatfunktion selbst als Namen ein).

* `SELECT COUNT(*) AS patient_count FROM d_patient`: liefert eine neue Spalte "patient_count", die die Gesamtzahl der Patienten enthält.

Häufiger Fehler: alle in einer solchen Abfrage nicht aggregierten Spalten *müssen* in der `GROUP BY`-Klausel aufgeführt werden, sonst kann das Statement nicht ausgeführt werden.

### Datensätze aggregieren

*Gesamtanzahl der Behandlungsfälle ausgeben:*

In [ ]:
%%sql $db_url_reporting
SELECT COUNT(*) AS fall_count
  FROM f_faelle

*Anzahl der eindeutigen Patienten-IDs in der Falltabelle ermitteln:*

In [ ]:
%%sql $db_url_reporting
SELECT COUNT(DISTINCT patient_id) AS patient_count
  FROM f_faelle

*Fallarten mit Fallzahl, mittlerer Liegedauer und Gesamterlös ausgeben:*

In [ ]:
%%sql $db_url_reporting
SELECT fallart_id,
       COUNT(*)                  AS fallzahl,
       AVG(liegedauer_tage)      AS mittlere_liegedauer,
       SUM(erloes_fallpauschale) AS gesamterloes
  FROM f_faelle
 GROUP BY fallart_id

### Filterung bei Aggregatfunktionen

Bei Aggregatfunktionen können während zwei verschiedenen Phasen der Abfrage Filterkriterien angewendet werden:
* die `WHERE`-Klausel wird auf die "Rohdaten" vor der Aggregation angewendet. Nur Datensätze, die die Filterkriterien erfüllen, werden bei der Aggregation einbezogen
* die `HAVING`-Klausel wird auf die bereits aggregierten Daten angewendet. So können Kriterien angewendet werden, die sich erst aus der Aggregation ergeben

Die `WHERE`-Klausel steht entsprechend vor der `GROUP BY`-Klausel, die `HAVING`-Klausel danach.

In der `HAVING`-Klausel muss die Aggregatfunktion in der Regel noch einmal explizit wiederholt werden, die Datenbank kann hier in der Regel nicht den vorher mit `AS` gesetzten Alias verwenden.

*Fallarten mit Fallzahl, mittlerer Liegedauer und Gesamterlös ausgeben, aber gefiltert:*
* *nur Fälle einbeziehen, die eine Liegedauer über 1 Tag hatten*
* *nur aggregierte Daten ausgeben, deren Fallzahl (pro Fallart) über 10 liegt*

In [ ]:
%%sql $db_url_reporting
SELECT fallart_id,
       COUNT(*)                  AS fallzahl,
       AVG(liegedauer_tage)      AS mittlere_liegedauer,
       SUM(erloes_fallpauschale) AS gesamterloes
  FROM f_faelle
 WHERE liegedauer_tage > 1
 GROUP BY fallart_id
HAVING COUNT(*) > 10

## Abfragen über mehrere Tabellen

Die Stärke relationaler Datenbanken liegt in der Möglichkeit, Tabellen über Primär- & Fremdschlüsselbeziehungen miteinander zu verknüpfen und gemeinsam abzufragen. Mit Hilfe der `JOIN`-Klausel können 2 oder mehr Tabellen miteinander verbunden werden. Hierbei müssen jeweils Kriterien angegeben werden, anhand derer die jeweils zusammengehörigen Datensätze erkannt werden können (typischerweise Primär- & Fremdschlüssel identischen Inhalts beider Tabellen).

Da es bei den in JOINs beteiligten Tabellen häufig identisch benannte Spalten gibt (z.B. die Spalte, über die der JOIN erfolgt), ist es in der Regel nötig, die Spalten durch Voranstellen des jeweiligen Tabellennamens zu qualifizieren (eindeutig zu machen). Da die Tabellennamen häufig lang sind, bietet es sich an, einen kurzen Alias für die Tabelle zu definieren, der statt des vollständigen Namens präfixiert werden kann.

Ein vollständiges `SELECT`-Statement mit `JOIN` sieht dann wie folgt aus:

`SELECT tb1.spalte1,
        tb2.spalte2
   FROM tabelle1 tb1
   JOIN tabelle2 tb2 ON tb1.spalte1 = tb2.spalte1`

"tb1" und "tb2" sind hierbei die Aliasnamen der beiden Tabellen.

Caveats:
* JOIN-Operationen können bei großen Tabellen sehr zeitaufwändig sein und große Last auf der Datenbank verursachen. Es sollte geprüft werden, ob z.B. Indizes auf den für JOINs verwendeten Spalten die Abfragen beschleunigen können. Viele Datenbanken bieten Befehle an, mit denen die Abfragestrategie der Datenbank nachvollzogen und Optimierungsmöglichkeiten erkannt werden können (für SQLite ÈXPLAIN QUERY PLAN`).
* Es sind immer JOIN-Kriterien zwischen den beteiligten Tabellen notwendig. Wenn zwei Tabellen ohne JOIN-Kriterien verbunden werden, entsteht ein *cartesian join*, d.h. die Datenbank verknüpft (mangels definierter Kriterien) einfach jeden Datensatz der ersten Tabelle mit allen Datensätzen der zweiten Tabelle. Das ist bis auf in seltenen Fällen unerwünscht.
* JOINs werden von der Datenbanken nacheinander abgearbeitet. JOIN-Kriterien müssen dabei immer nur von einer Tabelle zur "nächsten" angegeben werden. Es ist nicht nötig, alle Tabellen ("kreuz und quer") über Kriterien miteinander zu verbinden. Häufig gibt es tatsächlich auch nur zwischen jeweils "benachbarten" Tabellen passende Primär- & Fremdschlüssel (z.B. von der Patienten- auf die Falltabelle und von dort aus zur Diagnosetabelle, aber nicht zwischen der Diagnose- und Patiententabelle).

### INNER JOIN: Schnittmenge von Tabellen bilden

Standardfall ist der `INNER JOIN`: hierbei wird die Schnittmenge der beteiligten Tabellen gebildet, also nur für Datensätze, die beiden Tabellen identische Ausprägungen der für den JOIN verwendeten Spalten haben.

Im folgenden Beispiel werden die nicht sprechenden Fallart-IDs aus der Faktentabelle `F_FAELLE` um die Bezeichner aus der Dimensionstabelle `D_FALLART` ergänzt. Das Datenmodell sieht hier vor, dass es für jede verwendete Fallart-ID auch eine Entsprechung in der Dimensionstabelle gibt. Der INNER JOIN beider Tabellen sollte also alle Datensätze der Falltabelle einbeziehen.

*Fallarten mit Fallzahl, mittlerer Liegedauer und Gesamterlös ausgeben
(=> hier ergänzt um Text-Bezeichner der Fallart aus der zugehörigen Dimension):*

In [ ]:
%%sql $db_url_reporting
SELECT fal.fallart_id,
       frt.fallart_name,
       COUNT(*)                      AS fallzahl,
       AVG(fal.liegedauer_tage)      AS mittlere_liegedauer,
       SUM(fal.erloes_fallpauschale) AS gesamterloes
  FROM f_faelle  fal
  JOIN d_fallart frt ON fal.fallart_id = frt.fallart_id
 GROUP BY fal.fallart_id

### OUTER JOINs: Datensätze einbeziehen, für die es keine Entsprechung in beiden Tabellen gibt

In manchen Fällen ist es notwendig, auch Datensätze abzufragen, für die in der verbundenen Tabelle keine Daten vorliegen. Denkbar ist z.B. eine Patientenliste mit Hauptdiagnosen, die auch Patienten ausgibt, für die (noch) keine Hauptdiagnose dokumentiert wurde. Ggf. interessiert man sich auch *nur* für die Einträge ohne Bezug, z.B. um eine Arbeitsliste der noch nachzudokumentierenden Hauptdiagnosen zu bekommen.

Auch bei OUTER JOINs werden die Beziehungen nacheinander "von links nach rechts" abgearbeitet. Ein "LEFT OUTER JOIN" bezieht dabei alle Datensätze der "linken" Tabelle ein, und von der rechten nur die mit Bezug über die für den JOIN angegebenen Spalten. Der "RIGHT OUTER JOIN" funktioniert entsprechend umgekehrt und liefert alle Datensätze der "rechten" Tabelle sowie die passenden Datensätze der linken. Da es hierbei nur auf die Reihenfolge der Tabellen in der `FROM`-Klausel ankommt, beschränken sich manche Datenbanken (u.a. SQLite) auf den LEFT JOIN und überlassen den Nutzern, die Tabellen in die benötigte Reihenfolge zu bringen. Spalten, die in der "OUTER" hinzugezogenen Tabelle keine Entsprechung haben, werden als NULL-Werte zurückgegeben.

Beim FULL OUTER JOIN werden alle Datensätze beider Tabellen ausgegeben.

Caveat:
* Filterkriterien in der `WHERE`-Klausel, die auf Spalten der per OUTER JOIN verbundenen Tabelle gesetzt werden, können nur für Datensätze überprüft werden, die in dieser Tabelle auch befüllt werden. Da für Datensätze ohne Einträge in der verbundenen Tabelle nur NULL-Werte geliefert werden, schlagen die Überprüfungen fehl und Datensätze werden unerwartet ausgefiltert. Ein Workaround besteht darin, die Kriterien mit in die JOIN-Kriterien zu schreiben. Datensätze werden dann vor dem JOIN gefiltert, so dass Einträge ohne Verbindung erhalten bleiben. Einträge die diese Filterkriterien nicht erfüllen, werden natürlich weiterhin herausgefiltert (erscheinen dann aber mit NULL-Werten, obwohl sie in der verbundenen Tabelle enthalten waren).

*Fälle mit Bezeichnern für ihre Hauptdiagnosen ausgeben:*
* *hierzu wird die Dimensionstabelle D_DIAGNOSE per LEFT JOIN hinzugefügt*
* *Fälle ohne Hauptdiagnose werden hierbei weiterhin ausgegeben, für den Bezeichner der Diagnose wird aber NULL geliefert (hier im Jupyter Notebook als "None" angezeigt)*

In [ ]:
%%sql $db_url_reporting
SELECT fal.fall_id,
       fal.hauptdiagnose_snomed_id,
       dkt.snomed_name
  FROM      f_faelle fal
  LEFT JOIN d_diagnose dkt ON fal.hauptdiagnose_snomed_id = dkt.snomed_id
 LIMIT 10

*Fälle mit Bezeichnern für ihre Hauptdiagnosen ausgeben*
* *hier per Kriterium in der WHERE-Klausel gefiltert auf Diagnosen, die mit dem Wort 'Viral' beginnen*

*=> unerwünschterweise werden jetzt aber alle Fälle ausgeschlossen, für die keine Hauptdiagnose vorliegt, da das Filterkriterium mangels Daten in der verbundenen Tabelle nicht erfüllt werden kann*

In [ ]:
%%sql $db_url_reporting
SELECT fal.fall_id,
       fal.hauptdiagnose_snomed_id,
       dkt.snomed_name
  FROM      f_faelle fal
  LEFT JOIN d_diagnose dkt ON fal.hauptdiagnose_snomed_id = dkt.snomed_id
 WHERE dkt.snomed_name LIKE 'Viral%'
 LIMIT 10

*Fälle mit Bezeichnern für ihre Hauptdiagnosen ausgeben*
* *hier per Kriterium in der JOIN-Klausel gefiltert auf Diagnosen, die mit dem Wort 'Viral' beginnen*

*=> es bleiben alle Fälle aus der Tabelle F_FAELLE erhalten, auch wenn keine Diagnose dokumentiert wurde. Fälle, bei denen eine andere Diagnose (ohne "Viral" am Anfang) dokumentiert wurde, erhalten NULL-Werte aus der Diagnosedimension*

In [ ]:
%%sql $db_url_reporting
SELECT fal.fall_id,
       fal.hauptdiagnose_snomed_id,
       dkt.snomed_name
  FROM      f_faelle fal
  LEFT JOIN d_diagnose dkt ON fal.hauptdiagnose_snomed_id = dkt.snomed_id
                          AND dkt.snomed_name LIKE 'Viral%'
 LIMIT 10

UNIONs: Mehrere Tabellen "hintereinander" zusammenfügen

Während JOINs Tabellen "horizontal" verbinden, können mehrere Tabellen mit dem `UNION`-Statement "vertikal" hintereinander gestellt werden. `UNION` wird dabei nicht als Teil eines `SELECT`-Statements verwendet, sondern als Verbindung zwischen zwei oder mehr `SELECT`-Statements.

Die Ergebnisse der SELECT-Statements müssen dazu identische Spaltennamen und -datentypen haben.

Das `UNION`-Statement führt im Regelfall Datensätze, die mehrfach vorkommen (egal ob aus einer Tabelle oder über beide hinweg) zusammen, so wie es ein `DISTINCT`-Keyword in einem `SELECT`-Statement tun würde. Wenn alle Datensätze beider Tabellen benötigt werden, muss das `UNION ALL`-Statement verwendet werden.

*Datensätze für Diagnosen mit dem Wort "bronchitis" und "diabetes" zunächst getrennt
abfragen und dann über ein UNION-Statement zusammenfügen:*

In [ ]:
%%sql $db_url_reporting

SELECT *
  FROM d_diagnose
 WHERE snomed_name LIKE '%bronchitis%'

UNION

SELECT *
  FROM d_diagnose
 WHERE snomed_name LIKE '%diabetes%'